<a href="https://colab.research.google.com/github/ushifali/final-23/blob/master/Collaborative_Filtering_Attractions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd

# Load user preferences data
user_preferences_df  = pd.read_csv('/content/drive/MyDrive/Filtering_Data/user_preferences.csv', encoding='utf-8')

# Load attraction data
attractions_df = pd.read_csv('/content/drive/MyDrive/Filtering_Data/content_filtered_data_attraction.csv', encoding='utf-8')
attractions_df.head()

,Unnamed: 0.1,Unnamed: 0,address,city,description,latitude,longitude,price,state,street,title,totalScore,categories,image_url,Distance
0,274,274,"Murugeshpalya, Jeevan Bima Nagar, Bengaluru, K...",Bengaluru,NaN,12.961314,77.656118,NaN,Karnataka,NaN,GlobalTravellors,5.0,Tourist attraction,https://lh5.googleusercontent.com/p/AF1QipOOxD...,0.012503
1,239,239,"XMM8+X48, 1st Cross Rd, Bhuvaneswari Nagar, C ...",Bengaluru,NaN,12.984916,77.665327,NaN,Karnataka,"XMM8+X48, 1st Cross Rd",Shiva Statue,4.7,"Tourist attraction, Hindu temple",https://lh5.googleusercontent.com/p/AF1QipP5XR...,0.014535
2,257,257,"2-6, 9th Cross Rd, Kaggadasapura, Bengaluru, K...",Bengaluru,NaN,12.985504,77.675954,NaN,Karnataka,"2-6, 9th Cross Rd",Chinimika layout,5.0,Tourist attraction,NaN,0.018832
3,234,234,"3D, No, 3, 5th Cross Rd, Michael Palaya, H Col...",Bengaluru,NaN,12.981284,77.646522,NaN,Karnataka,"3D, No, 3, 5th Cross Rd",Freedom Wall Bangalore,4.7,Tourist attraction,https://lh5.googleusercontent.com/p/AF1QipNt7G...,0.021202
4,236,236,"Police Station, Near HAL, HAL Old Airport Rd, ...",Bengaluru,Aerospace company's museum featuring photograp...,12.954655,77.680906,NaN,Karnataka,"Police Station, Near HAL, HAL Old Airport Rd",HAL Heritage Centre and Aerospace Museum,4.4,"Tourist attraction, Heritage museum, Museum",https://lh5.googleusercontent.com/p/AF1QipNb68...,0.022581


In [5]:

from sklearn.metrics.pairwise import pairwise_distances
from sklearn.preprocessing import MultiLabelBinarizer


# Define a function to calculate Jaccard similarity between a user's preferences and all restaurants
def calculate_similarity(user_id):
    user_prefs = user_preferences_df[user_preferences_df['user_id'] == user_id].iloc[:, 3:].values
    attraction_features = attractions_df.iloc[:, 5:].values

    # Create a MultiLabelBinarizer object to one-hot encode the values in each column as binary features
    mlb = MultiLabelBinarizer()

    # Fit the MultiLabelBinarizer on all the values in the 'Cuisines' column of the restaurants dataframe
    mlb.fit(attractions_df['categories'].str.split(', '))

    # Transform the 'Cuisines' column of both user preferences and restaurants dataframes into binary features
    user_categories = mlb.transform(user_preferences_df[user_preferences_df['user_id'] == user_id]['categories'].str.split(', '))
    attraction_categories = mlb.transform(attractions_df['categories'].str.split(', '))

    # Calculate Jaccard similarity between the user's preferences and all restaurants
    similarities = pairwise_distances(user_categories, attraction_categories, metric='jaccard')

    return 1 - similarities[0]





In [6]:
attraction_names = attractions_df['title']

In [7]:
# Example usage: Recommend top 5 restaurants for user 1
user_id = 1
similarities = calculate_similarity(user_id)
attraction_data = attractions_df[['title', 'categories', 'address', 'city', 'totalScore' ,'Distance']]
recommendations = pd.DataFrame({'title': attraction_names, 'Similarity': similarities})
recommendations = recommendations.merge(attraction_data, on='title').sort_values(by='Similarity', ascending=False).head(5)


/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) [' Amusement center', ' Amusement park', ' Animal park', ' Auditorium', ' Botanical garden', ' Cafe', ' Chapel', " Children's museum", ' Dog park', ' Escape room center', ' Event management company', ' Forestry service', ' Fortress', ' Garden', ' Heritage building', ' Heritage museum', ' Hindu temple', ' Historical landmark', ' Historical place', ' Historical place museum', ' Indoor snowcenter', ' Landmark', ' Learning center', ' Memorial park', ' Museum', ' National museum', ' National park', ' Park', ' Pet adoption service', ' Place of worship', ' Planetarium', ' Playground', ' Public beach', ' Public library', ' Sculpture', ' Shopping mall', ' Software company', ' Souvenir store', ' Swimming lake', ' Technology museum', ' Tourist attraction', ' Wildlife and safari park'] will be ignored
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: Da

In [8]:
print(recommendations.to_string(index=False))
recommendations.head()

                         title  Similarity               categories                                                                                                                address      city  totalScore  Distance
                    Car Park 🚗    0.333333                     Park                         VJP6+MH2, 5th Main Rd, 2nd Block, BDA Layout BTM 4th Stage, Bengaluru, Karnataka 560076, India Bengaluru         4.5  0.099294
                Childrens Park    0.250000 Tourist attraction, Park                                                     WJ7Q+CHQ, Sector 7, HSR Layout, Bengaluru, Karnataka 560102, India Bengaluru         4.0  0.062372
         BBMP Park Koramangala    0.250000 Tourist attraction, Park WJ9P+8JQ, 13th Cross Rd, Jakkasandra, 1st Block Koramangala, HSR Layout 5th Sector, Bengaluru, Karnataka 560034, India Bengaluru         4.0  0.059153
               Tavarekere Park    0.250000 Tourist attraction, Park                                            Tank Bund Rd,

,title,Similarity,categories,address,city,totalScore,Distance
125,Car Park 🚗,0.333333,Park,"VJP6+MH2, 5th Main Rd, 2nd Block, BDA Layout B...",Bengaluru,4.5,0.099294
44,Childrens Park,0.250000,"Tourist attraction, Park","WJ7Q+CHQ, Sector 7, HSR Layout, Bengaluru, Kar...",Bengaluru,4.0,0.062372
40,BBMP Park Koramangala,0.250000,"Tourist attraction, Park","WJ9P+8JQ, 13th Cross Rd, Jakkasandra, 1st Bloc...",Bengaluru,4.0,0.059153
73,Tavarekere Park,0.250000,"Tourist attraction, Park","Tank Bund Rd, Bhavani Nagar, S.G. Palya, Benga...",Bengaluru,4.2,0.075075
27,Seetharampalya Lake View Point,0.250000,"Tourist attraction, Park","XPJ6+PX9, Seetharampalya, Hoodi, Bengaluru, Ka...",Bengaluru,3.9,0.049125


In [9]:
# Example usage: Recommend top 5 restaurants for user 5
user_id = 7
similarities = calculate_similarity(user_id)
attraction_data = attractions_df[['title', 'categories', 'address', 'city', 'totalScore' ,'Distance']]
recommendations = pd.DataFrame({'title': attraction_names, 'Similarity': similarities})
recommendations = recommendations.merge(attraction_data, on='title').sort_values(by='Similarity', ascending=False).head(5)
print(recommendations.to_string(index=False))
recommendations.head()

                title  Similarity                        categories                                                                                                     address      city  totalScore  Distance
           Car Park 🚗    0.333333                              Park              VJP6+MH2, 5th Main Rd, 2nd Block, BDA Layout BTM 4th Stage, Bengaluru, Karnataka 560076, India Bengaluru         4.5  0.099294
Time For Travel Tours    0.250000 Tour operator, Tourist attraction       No 734 Ground Floor, 1st B Cross Rd, 3rd Stage, Basaveshwar Nagar, Bengaluru, Karnataka 560079, India Bengaluru         5.0  0.128688
  Mahatma Gandhi Park    0.250000          Tourist attraction, Park                                                 XHGX+VV9, Shivaji Nagar, Bengaluru, Karnataka 560001, India Bengaluru         4.1  0.065334
Buddha Shanthi Kanive    0.250000          Tourist attraction, Park XGH7+VC9, 7th Cross Rd, 3rd Block, BDA Layout, 2nd Stage, Naagarabhaavi, Bengaluru, Karnataka 560072

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) [' Animal park', ' Auditorium', ' Bird watching area', ' Cafe', ' Catholic church', " Children's party service", ' Event management company', ' Heritage building', ' Heritage museum', ' Hindu temple', ' Historical place', ' Indoor snowcenter', ' Memorial park', ' Mosque', ' Park', ' Pet adoption service', ' Place of worship', ' Planetarium', ' Research institute', ' Sculpture', ' Shopping mall', ' Tourist attraction', ' Wildlife and safari park'] will be ignored
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


,title,Similarity,categories,address,city,totalScore,Distance
125,Car Park 🚗,0.333333,Park,"VJP6+MH2, 5th Main Rd, 2nd Block, BDA Layout B...",Bengaluru,4.5,0.099294
159,Time For Travel Tours,0.250000,"Tour operator, Tourist attraction","No 734 Ground Floor, 1st B Cross Rd, 3rd Stage...",Bengaluru,5.0,0.128688
49,Mahatma Gandhi Park,0.250000,"Tourist attraction, Park","XHGX+VV9, Shivaji Nagar, Bengaluru, Karnataka ...",Bengaluru,4.1,0.065334
179,Buddha Shanthi Kanive,0.250000,"Tourist attraction, Park","XGH7+VC9, 7th Cross Rd, 3rd Block, BDA Layout,...",Bengaluru,4.4,0.151385
258,tannirubavi tree park,0.250000,"Tourist attraction, Park","VRF9+WGP, Sultan Battery, Sultan Battery Road,...",Mangaluru,4.3,2.847485


In [10]:
# Example usage: Recommend top 5 restaurants for user 20
user_id = 37
similarities = calculate_similarity(user_id)
attraction_data = attractions_df[['title', 'categories', 'address', 'city', 'totalScore' ,'Distance']]
recommendations = pd.DataFrame({'title': attraction_names, 'Similarity': similarities})
recommendations = recommendations.merge(attraction_data, on='title').sort_values(by='Similarity', ascending=False).head(5)
print(recommendations.to_string(index=False))
recommendations.head()

                title  Similarity                          categories                                                                                                              address        city  totalScore  Distance
Time For Travel Tours         0.5   Tour operator, Tourist attraction                No 734 Ground Floor, 1st B Cross Rd, 3rd Stage, Basaveshwar Nagar, Bengaluru, Karnataka 560079, India   Bengaluru         5.0  0.128688
Chikkabettahalli Lake         0.0                  Tourist attraction                                          3HR3+X85, Adityanagar, Vidyaranyapura, Rajahmundry, Karnataka 560097, India Rajahmundry         3.7  0.165189
              Girisha         0.0                  Tourist attraction No 20, ,SRS road, 2 nd cross post, Ganigarapalya, Adarsha Layout, Talaghattapura, Bengaluru, Karnataka 560062, India   Bengaluru         2.5  0.164724
         Stone quarry         0.0                  Tourist attraction                                            3HV

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) [' Amusement center', ' Cafe', ' Cathedral', ' Chapel', ' Christian church', ' Church', ' Education center', ' Fortress', ' Garden', ' Heritage museum', ' Hindu temple', ' Historical landmark', ' National museum', ' Pet adoption service', ' Place of worship', ' Playground', ' Science museum', ' Software company', ' Swimming lake', ' Technology museum', ' Tourist attraction'] will be ignored
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


,title,Similarity,categories,address,city,totalScore,Distance
159,Time For Travel Tours,0.5,"Tour operator, Tourist attraction","No 734 Ground Floor, 1st B Cross Rd, 3rd Stage...",Bengaluru,5.0,0.128688
195,Chikkabettahalli Lake,0.0,Tourist attraction,"3HR3+X85, Adityanagar, Vidyaranyapura, Rajahmu...",Rajahmundry,3.7,0.165189
194,Girisha,0.0,Tourist attraction,"No 20, ,SRS road, 2 nd cross post, Ganigarapal...",Bengaluru,2.5,0.164724
193,Stone quarry,0.0,Tourist attraction,"3HV3+4F8, Adityanagar, Vidyaranyapura, Bengalu...",Bengaluru,1.0,0.164693
192,Turahalli Forest,0.0,"Tourist attraction, National forest","VGMF+FXQ, Thurahalli, Bengaluru, Karnataka 560...",Bengaluru,4.3,0.164438
